In [1]:
import numpy as np

### 50.000 combinaciones de lotería. Objetivo: Análisis de rendimiento del programa.

___

In [19]:
combinaciones = 50000
n_bolas = 5

### Solo usando modulos del lenguaje 

In [9]:
import random

In [10]:
def sacar_bola_slow(combi, n_bola):
    "esta forma pued creo que puede producir bolas repetidas"
    bola = random.randint(0, 50)
    for bola_comp in range(n_bola):
        if bola == combi[bola_comp]:
            bola = random.randint(0, 50)
    return bola

In [11]:
%%time
combi_ganadora = []
for i in range(n_bolas):
    combi_ganadora.append(sacar_bola_slow(combi_ganadora, i))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(n_bolas):
        combi_apostada.append(sacar_bola_slow(combi_apostada, n_bola))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = 0
    for bola_ganadora in combi_ganadora:
        for bola_apostada in combi_apostada:
            if bola_ganadora == bola_apostada:
                aciertos_combinacion += 1
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(n_bolas):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

0 : 29229
1 : 17303
2 : 3228
3 : 235
4 : 5
CPU times: user 1.47 s, sys: 40.7 ms, total: 1.51 s
Wall time: 1.57 s


### Podemos mejorar la funcion sacar bola con recursion

In [14]:
def sacar_bola(combi):
    "usa recursion, mo produce repetias"
    bola = random.randint(0, 50)
    if bola in combi:
        bola = sacar_bola(combi)
    return bola

In [ ]:
%%time
combi_ganadora = []
for i in range(n_bolas):
    combi_ganadora.append(sacar_bola(combi_ganadora))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(n_bolas):
        combi_apostada.append(sacar_bola(combi_apostada))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = 0
    for bola_ganadora in combi_ganadora:
        for bola_apostada in combi_apostada:
            if bola_ganadora == bola_apostada:
                aciertos_combinacion += 1
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(n_bolas):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

#### Podemos sustituir por sets los dos bucles del final

In [21]:
%%time
combi_ganadora = []
for i in range(5):
    combi_ganadora.append(sacar_bola(combi_ganadora))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(5):
        combi_apostada.append(sacar_bola(combi_apostada))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = len(set(combi_apostada).intersection(combi_ganadora))
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(5):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

0 : 29077
1 : 17434
2 : 3276
3 : 204
4 : 9
CPU times: user 1.07 s, sys: 15 ms, total: 1.08 s
Wall time: 1.14 s


### Con numpy

In [22]:
import numpy as np

In [23]:
%%time
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

aciertos = np.zeros(combinaciones)

for i in range(combinaciones):
    combinacion = np.random.choice(50, 5, replace=True)
    combi_apostada = np.zeros(50)
    combi_apostada[combinacion] = 1
    
    aciertos[i] = combi_ganadora@combi_apostada

unique, counts = np.unique(aciertos, return_counts=True)
print(f"{unique} {counts}")

[0. 1. 2. 3. 4.] [29524 17142  3142   186     6]
CPU times: user 2.04 s, sys: 49.5 ms, total: 2.08 s
Wall time: 2.07 s


Podemos definir las apuestas antes:

In [30]:
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas[:, :5] = 1
[np.random.shuffle(apuesta) for apuesta in apuestas]

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)
print(f"{unique} {counts}")

[0. 1. 2. 3. 4.] [28690 17630  3430   246     4]


aun así sigue siendo lento

### Lo mas eficiente que se puede hacer con numpy es de la siguiente forma:

In [47]:
index = np.tile(np.array([range(combinaciones)]).transpose(), (1, 5))

In [48]:
index

array([[    0,     0,     0,     0,     0],
       [    1,     1,     1,     1,     1],
       [    2,     2,     2,     2,     2],
       ...,
       [49997, 49997, 49997, 49997, 49997],
       [49998, 49998, 49998, 49998, 49998],
       [49999, 49999, 49999, 49999, 49999]])

In [50]:
np.array([range(combinaciones)]).shape

(1, 50000)

In [54]:
?np.arange

Docstring:
arange([start,] stop[, step,], dtype=None)

Return evenly spaced values within a given interval.

Values are generated within the half-open interval ``[start, stop)``
(in other words, the interval including `start` but excluding `stop`).
For integer arguments the function is equivalent to the Python built-in
`range` function, but returns an ndarray rather than a list.

When using a non-integer step, such as 0.1, the results will often not
be consistent.  It is better to use `numpy.linspace` for these cases.

Parameters
----------
start : number, optional
    Start of interval.  The interval includes this value.  The default
    start value is 0.
stop : number
    End of interval.  The interval does not include this value, except
    in some cases where `step` is not an integer and floating point
    round-off affects the length of `out`.
step : number, optional
    Spacing between values.  For any output `out`, this is the distance
    between two adjacent values, ``out[i+1]

In [56]:
.shape

(1, 50000)

In [53]:
.shape

(50000,)

In [59]:
%%time
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas_num = np.random.rand(combinaciones, 50).argpartition(5,axis=1)[:,:5]
index = np.tile(np.expand_dims(np.arange(combinaciones), axis=0).transpose(), (1, 5))
apuestas[index, apuestas_num] = 1

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)

CPU times: user 109 ms, sys: 19.1 ms, total: 128 ms
Wall time: 127 ms


In [44]:
print(f"{unique} {counts}")

[0. 1. 2. 3. 4.] [28926 17546  3304   221     3]


Podemos medirlo varias veces

In [ ]:
%%timeit
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas_num = np.random.rand(combinaciones, 50).argpartition(5,axis=1)[:,:5]
index = np.tile(np.expand_dims(np.arange(combinaciones), axis=0).transpose(), (1, 5))
apuestas[index, apuestas_num] = 1

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)

133ms en mi ordenador aprox